In [2]:
!pip install iterative-stratification

  Using cached iterative_stratification-0.1.6-py3-none-any.whl (8.7 kB)


In [3]:
import pandas as pd
import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [5]:
train_labels = pd.read_csv("/home/ubuntu/datasets/human-protein-atlas/external_data_raw/train.csv")
train_labels.head()

,Id,Target
0,10580_1610_C1_1,13 25 0 2 21
1,10580_1610_C1_2,13 25 0 2 21
2,10580_1756_B1_1,13 25 0 2 21
3,10580_1756_B1_2,13 25 0 2 21
4,10580_1758_B1_1,13 25 0 2 21


In [6]:
# from sklearn.preprocessing import MultiLabelBinarizer

# one_hot = MultiLabelBinarizer()

# # One-hot encode data
# y = one_hot.fit_transform(df["Target"])

In [7]:
label_names = {
    0:  "Nucleoplasmn",  
    1:  "Nuclear membrane",   
    2:  "Nucleoli",   
    3:  "Nucleoli fibrillar center",   
    4:  "Nuclear speckles",
    5:  "Nuclear bodies",   
    6:  "Endoplasmic reticulum",   
    7:  "Golgi apparatus",   
    8:  "Peroxisomes",   
    9:  "Endosomes",   
    10:  "Lysosomes", 
    11:  "Intermediate filaments",   
    12:  "Actin filaments",   
    13:  "Focal adhesion sites",   
    14:  "Microtubules",   
    15:  "Microtubule ends",   
    16:  "Cytokinetic bridge",   
    17:  "Mitotic spindle",   
    18:  "Microtubule organizing center",   
    19:  "Centrosome",   
    20:  "Lipid droplets",   
    21:  "Plasma membrane",   
    22:  "Cell junctions",   
    23:  "Mitochondria",   
    24:  "Aggresome",   
    25:  "Cytosol",   
    26:  "Cytoplasmic bodies",   
    27:  "Rods & rings"
}

In [8]:
reverse_train_labels = dict((v,k) for k,v in label_names.items())

def fill_targets(row):
    row.Target = np.array(row.Target.split(" ")).astype(np.int)
    for num in row.Target:
        name = label_names[int(num)]
        row.loc[name] = 1
    return row

In [9]:
for key in label_names.keys():
    train_labels[label_names[key]] = 0

train_labels = train_labels.apply(fill_targets, axis=1)

In [10]:
train_labels.head()

,Id,Target,Nucleoplasmn,Nuclear membrane,Nucleoli,Nucleoli fibrillar center,Nuclear speckles,Nuclear bodies,Endoplasmic reticulum,Golgi apparatus,...,Microtubule organizing center,Centrosome,Lipid droplets,Plasma membrane,Cell junctions,Mitochondria,Aggresome,Cytosol,Cytoplasmic bodies,Rods & rings
0,10580_1610_C1_1,"[13, 25, 0, 2, 21]",1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,10580_1610_C1_2,"[13, 25, 0, 2, 21]",1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,10580_1756_B1_1,"[13, 25, 0, 2, 21]",1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,10580_1756_B1_2,"[13, 25, 0, 2, 21]",1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,10580_1758_B1_1,"[13, 25, 0, 2, 21]",1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [11]:
train_labels.shape

(74606, 30)

In [12]:
train_labels.loc[:,"kfold"] = -1

In [13]:
train_labels.columns

Index(['Id', 'Target', 'Nucleoplasmn', 'Nuclear membrane', 'Nucleoli',
       'Nucleoli fibrillar center', 'Nuclear speckles', 'Nuclear bodies',
       'Endoplasmic reticulum', 'Golgi apparatus', 'Peroxisomes', 'Endosomes',
       'Lysosomes', 'Intermediate filaments', 'Actin filaments',
       'Focal adhesion sites', 'Microtubules', 'Microtubule ends',
       'Cytokinetic bridge', 'Mitotic spindle',
       'Microtubule organizing center', 'Centrosome', 'Lipid droplets',
       'Plasma membrane', 'Cell junctions', 'Mitochondria', 'Aggresome',
       'Cytosol', 'Cytoplasmic bodies', 'Rods & rings', 'kfold'],
      dtype='object')

In [14]:
# random shuffle with fraction = 1 then reset the index
train_labels = train_labels.sample(frac=1).reset_index(drop=True)

X = train_labels.Id.values
#y = df[["grapheme_root","vowel_diacritic","consonant_diacritic"]].values
y = train_labels.iloc[:,2:].values

# define how many folds
mskf = MultilabelStratifiedKFold(n_splits=5)

In [15]:
X[:5], y[:5]

(array(['71224_1339_E6_1', '35162_1558_G7_1', '42907_1134_E12_2',
        '22329_656_E1_2', '40366_754_E4_5'], dtype=object),
 array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0, -1],
        [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0, -1],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0, -1],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0, -1]]))

In [16]:
# Let's go through the iterator and get train and val indices
for fold, (trn_, val_) in enumerate(mskf.split(X, y)):
    print("TRAIN: ", trn_, "VALID : ",val_)
    train_labels.loc[val_, "kfold"] = fold

TRAIN:  [    0     1     2 ... 74602 74604 74605] VALID :  [    6    36    38 ... 74596 74600 74603]
TRAIN:  [    0     1     2 ... 74603 74604 74605] VALID :  [    4     7    11 ... 74591 74592 74598]
TRAIN:  [    0     1     2 ... 74603 74604 74605] VALID :  [    3     5     9 ... 74597 74601 74602]
TRAIN:  [    1     3     4 ... 74601 74602 74603] VALID :  [    0     2     8 ... 74599 74604 74605]
TRAIN:  [    0     2     3 ... 74603 74604 74605] VALID :  [    1    12    16 ... 74588 74589 74590]


In [17]:
train_labels["kfold"]

0        3
1        4
2        3
3        2
4        1
        ..
74601    2
74602    2
74603    0
74604    3
74605    3
Name: kfold, Length: 74606, dtype: int64

In [18]:
train_labels["kfold"].value_counts()

3    14922
4    14921
2    14921
1    14921
0    14921
Name: kfold, dtype: int64

Let's check a few values to ensure the splitting went as expected

In [19]:
# Common category
train_labels.groupby('kfold')['Nucleoplasmn'].sum()

kfold
0    5559
1    5652
2    5606
3    5606
4    5650
Name: Nucleoplasmn, dtype: int64

In [20]:
train_labels.groupby('kfold')['Nucleoli'].sum()

kfold
0    1450
1    1450
2    1450
3    1450
4    1450
Name: Nucleoli, dtype: int64

In [21]:
# Rare category
train_labels.groupby('kfold')['Peroxisomes'].sum()

kfold
0    33
1    33
2    32
3    33
4    33
Name: Peroxisomes, dtype: int64

In [22]:
# Rare category
train_labels.groupby('kfold')['Aggresome'].sum()

kfold
0    21
1    21
2    22
3    21
4    21
Name: Aggresome, dtype: int64

__^ Looks like the classes got equally distributed in all folds as expected.__

In [23]:
train_labels.to_csv("../input/train_folds_external_data.csv", index=False)